In [377]:
import numpy as np
import pandas as pd
import requests
from io import StringIO
from bs4 import BeautifulSoup as BS
from time import sleep
from tqdm import tqdm
import re

In [194]:
fda_df = pd.read_csv('Data/fdalabel-query-327575.csv')

In [195]:
fda_df.head(10)

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
0,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,ORAL,NDA,212690,XYWAV,"(CALCIUM, MAGNESIUM, POTASSIUM, AND SODIUM OXYBATES)",20210823,Central Nervous System Depressant,2002.0,...,2020/11/02-,68727-150,7G33012534; 8W24SYD6ZI; G983HLV265; S8NKF3H3KT,Calcium Oxybate; Magnesium Oxybate; Potassium Oxybate; SODIUM OXYBATE,4-HYDROXYBUTANOIC ACID,30IW36W5B2,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1e0ae43a-037f-42af-8e23-a0e51d75abe8/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=1e0ae43a-037f-42af-8e23-a0e51d75abe8,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=1e0ae43a-037f-42af-8e23-a0e51d75abe8,1e0ae43a-037f-42af-8e23-a0e51d75abe8
1,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INFILTRATION; PERINEURAL,NDA,009435,(CHLOROPROCAINE HCI,(CHLOROPROCAINE HCI,20200222,Ester Local Anesthetic,NaN,...,2019/10/13-,51662-1407,LT7Z1YW11H,CHLOROPROCAINE HYDROCHLORIDE,CHLOROPROCAINE; EDETIC ACID,5YVB0POT2H; 9G34HU7RV0,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/94d13f48-fbca-67b4-e053-2995a90a791b/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=94d13f48-fbca-67b4-e053-2995a90a791b,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=94d13f48-fbca-67b4-e053-2995a90a791b,94d13f48-fbca-67b4-e053-2995a90a791b
2,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,AURICULAR (OTIC),NDA,208251,OTOVEL,(CIPROFLOXACIN AND FLUOCINOLONE ACETONIDE),20190912,Corticosteroid; Quinolone Antimicrobial,2016.0,...,2016/04/29-,24338-080,0CD5FD6S2M; 5E8K9I0O4U,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,0CD5FD6S2M; 5E8K9I0O4U,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1a7e4806-3138-4b7a-b798-a99e163ee846/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=1a7e4806-3138-4b7a-b798-a99e163ee846,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=1a7e4806-3138-4b7a-b798-a99e163ee846,1a7e4806-3138-4b7a-b798-a99e163ee846
3,HUMAN PRESCRIPTION DRUG LABEL,KIT,INTRAVENOUS,BLA,125611,REBINYN,"(COAGULATION FACTOR IX (RECOMBINANT), GLYCOPEGYLATED)",20200630,Human Blood Coagulation Factor,2017.0,...,2017/12/01-,0169-7901; 0169-7902; 0169-7905,382L14738L,COAGULATION FACTOR IX RECOMBINANT HUMAN,COAGULATION FACTOR IX RECOMBINANT HUMAN,382L14738L,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/0ea37235-35fd-410d-b8c4-40ba15fe1294/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=0ea37235-35fd-410d-b8c4-40ba15fe1294,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=0ea37235-35fd-410d-b8c4-40ba15fe1294,0ea37235-35fd-410d-b8c4-40ba15fe1294
4,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION",INTRAVENOUS,NDA,209401,VYXEOS,(DAUNORUBICIN AND CYTARABINE) LIPOSOME,20210413,Anthracycline Topoisomerase Inhibitor; Nucleoside Metabolic Inhibitor,2017.0,...,2017/08/03-,68727-745,04079A1RDZ; ZS7284E0ZP,CYTARABINE; DAUNORUBICIN,CYTARABINE; DAUNORUBICIN,04079A1RDZ; ZS7284E0ZP,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7ea701ce-e7d3-4349-a9c2-642a501d45c8/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=7ea701ce-e7d3-4349-a9c2-642a501d45c8,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=7ea701ce-e7d3-4349-a9c2-642a501d45c8,7ea701ce-e7d3-4349-a9c2-642a501d45c8
5,HUMAN PRESCRIPTION DRUG LABEL,INJECTION,INTRAVENOUS,BLA,761107,GAMIFANT,(EMAPALUMAB-LZSG),20200630,Interferon gamma Blocker,2018.0,...,2019/05/17-; 2020/06/26-,66658-501; 66658-505; 66658-510,3S252O2Z4X,emapalumab,emapalumab,3S252O2Z4X,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/a865e0ef-8685-4f69-8838-648c4f3ba

In [196]:
fda_df['Trade Name'] = fda_df['Trade Name'].str.lower()

In [197]:
fda_df = fda_df.sort_values('Most Recent SPL Date (YYYY/MM/DD)', ascending = False)

In [198]:
fda_df.drop_duplicates(subset='Trade Name')

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
10855,HUMAN PRESCRIPTION DRUG LABEL,TABLET,ORAL,Export only,NaN,cyproheptadine hydrochloride,CYPROHEPTADINE HYDROCHLORIDE,2010110126,NaN,NaN,...,2010/06/15-,62033-0346,NJ82J0F8QC,Cyproheptadine Hydrochloride,Cyproheptadine,2YHB6175DO,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,f90faaa3-f40f-4653-8fb8-144d61c6b1d4
17740,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,077525,gabapentin,GABAPENTIN,20220321,Anti-epileptic Agent,1993.0,...,2006/08/24-,71610-069,6CW7F3G59X,GABAPENTIN,GABAPENTIN,6CW7F3G59X,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=7790d906-af26-4e60-bc6f-e906ec03cc24,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=7790d906-af26-4e60-bc6f-e906ec03cc24,7790d906-af26-4e60-bc6f-e906ec03cc24
22866,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,078106,levetiracetam,LEVETIRACETAM,20220108,NaN,1999.0,...,2012/09/11-,43547-221; 43547-222; 43547-223; 43547-224,44YRR34555,LEVETIRACETAM,LEVETIRACETAM,44YRR34555,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=dd713e60-478c-43cd-b680-ff594a0b3864,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=dd713e60-478c-43cd-b680-ff594a0b3864,dd713e60-478c-43cd-b680-ff594a0b3864
5728,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,ANDA,077415,bupropion hydrochloride,BUPROPION HYDROCHLORIDE,20220101,Aminoketone,1985.0,...,2009/05/01-,63187-713,ZG7E5POY8O,BUPROPION HYDROCHLORIDE,BUPROPION,01ZG3TPX31,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=e57451fd-aba5-47d6-9892-ffe2a6021b6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=e57451fd-aba5-47d6-9892-ffe2a6021b6f,e57451fd-aba5-47d6-9892-ffe2a6021b6f
17439,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION",INTRAVENOUS,ANDA,204025,fosaprepitant,FOSAPREPITANT,20211231,Substance P/Neurokinin-1 Receptor Antagonist,2008.0,...,2020/10/22-,16729-240,D35FM8T64X,FOSAPREPITANT DIMEGLUMINE,APREPITANT,1NF15YR6UY,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=4a3bac1c-1702-4963-b967-f1699f03cc6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=4a3bac1c-1702-4963-b967-f1699f03cc6f,4a3bac1c-1702-4963-b967-f1699f03cc6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33647,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,NaN,NaN,urobiotic,"OXYTETRACYCLINE HYDROCHLORIDE, SULFAMETHIZOLE AND PHENAZOPYRIDINE HYDROCHLORIDE",20051214,NaN,NaN,...,NaN,0049-0920,25W8454H16; 2IUY41693Z; 4U7K4N52ZM,oxytetracycline hydrochloride; phenazopyridine hydrochloride; sulfamethizole,oxytetracycline; phenazopyridine; sulfamethizole,25W8454H16; X20I9EN955,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/be27854a-a805-4300-9729-accd1b7f226f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=be27854a-a805-4300-9729-accd1b7f226f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=be27854a-a805-4300-9729-accd1b7f226f,be27854a

In [199]:
pd.set_option('display.max_colwidth', None)
fda_df['FDALabel Link']

10855    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc
17740    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc
22866    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc
5728     http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc
17439    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc
                                                        ...                                                
26675    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40e35fa9-9cfe-04f1-e054-00144ff8d46c/spl-doc
40691    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1cd07b54-f66f-4040-9058-462324267196/spl-doc
22289    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40d145e6-13f7-41ac-e054-00144ff8d46c/spl-doc
36580    http://nctr-crs.fda

In [473]:
drugz = 'levetiracetam'

In [474]:
endpoint = f'https://www.drugs.com/sfx/{drugz}-side-effects.html'

In [475]:
response = requests.get(endpoint)

In [476]:
response

<Response [200]>

In [477]:
soup = BS(response.text)

In [204]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Gabapentin Side Effects: Common, Severe, Long Term - Drugs.com
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Learn about the potential side effects of gabapentin. Includes common and rare side effects information for consumers and healthcare professionals." name="description"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <link href="https://www.drugs.com/sfx/gabapentin-side-effects.html" rel="canonical"/>
  <link href="/opensearch.xml" rel="search" title="Drugs.com" type="application/opensearchdescription+xml"/>
  <!-- Facebook Open Graph Summary -->
  <meta content="1470988726468809" property="fb:app_id"/>
  <meta content="https://www.facebook.com/Drugscom" property="article:author"/>
  <meta content="https://www.facebook.com/Drugscom" property="article:publisher"/>
  <meta content="Gabapentin Side Effects: Common, Severe, Long Term -

In [424]:
#COME BACK TO THIS TO SEE IF ITS POSSIBLE. THE WAY TO DO IT WOULD BE GRAB FIRST 2 H3, THEN GRAB THE Is and put them into LIST,
#THEN GRAB LIs UNDER EACH I AND PUT THEM INTO A LIST, THEN ZIP LISTS 
# list1 = []
# for span in soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True):
#      text = span.next_element

#      if text:
#          list1.append(span)

        
# x = list1.index('More common')
# y = list1.index('General')
# list1[x:y]
        
        

In [478]:
list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)

In [432]:
s

[<li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="a" href="/alpha/a.html">a</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="b" href="/alpha/b.html">b</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="c" href="/alpha/c.html">c</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="d" href="/alpha/d.html">d</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="e" href="/alpha/e.html">e</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="f" href="/alpha/f.html">f</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="g" href="/alpha/g.html">g</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="h" href="/alpha/h.html">h</a></li>,
 <li><a data-ga-action="Alpha" data-ga-category="Top Nav" data-ga-label="i" href="/alpha/i.html">i</a></li>,
 <li><a data-ga-act

In [479]:
str_list1 = [str(i) for i in list1]

In [481]:
index1 = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')

In [482]:
index2 = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])

In [454]:
q = list1[index1:index2][0]

In [456]:
q.name


'h3'

In [483]:
h3 = ''
i = ''
li = ''
results = []
for ele in list1[index1:index2]:
    if ele.name == 'h3':
        h3 = ele.text
    elif ele.name == 'i':
        i = ele.text
    else:
        li = ele.text
        results.append((drugz,h3,i,li))

In [484]:
pd.DataFrame(results)

,0,1,2,3
0,levetiracetam,Side effects requiring immediate medical attention,More common,Aggressive or angry
1,levetiracetam,Side effects requiring immediate medical attention,More common,anxiety
2,levetiracetam,Side effects requiring immediate medical attention,More common,change in personality
3,levetiracetam,Side effects requiring immediate medical attention,More common,chills
4,levetiracetam,Side effects requiring immediate medical attention,More common,cough
...,...,...,...,...
113,levetiracetam,Side effects not requiring immediate medical attention,Less common,"burning, dry, or itching eyes"
114,levetiracetam,Side effects not requiring immediate medical attention,Less common,change in the color of the skin
115,levetiracetam,Side effects not requiring immediate medical attention,Less common,congestion
116,levetiracetam,Side effects not requiring immediate medical attention,Less common,cough increased


In [205]:
side_efx = soup.find('p', attrs = {'class':False})

In [206]:
side_efx = side_efx.text

In [207]:
side_efx = side_efx.split('.')

In [208]:
side_efx[0:2]
common_se = side_efx[0]
other_se = side_efx[1]

In [209]:
common_se = common_se.split(':')
other_se = other_se.split(':')

In [210]:
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [213]:
common_se_dict = Convert(common_se)
other_se_dict = Convert(other_se)

In [219]:
common_se_dict

{'Commonly reported side effects of gabapentin include': ' ataxia, dizziness, drowsiness, fatigue, fever, nystagmus disorder, sedated state, and viral infection',
 ' Other side effects include': ' blurred vision, diplopia, peripheral edema, tremor, amblyopia, irritability, and xerostomia'}

In [239]:
fda_df.head()

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID,webscraping_name
10855,HUMAN PRESCRIPTION DRUG LABEL,TABLET,ORAL,Export only,NaN,cyproheptadine hydrochloride,CYPROHEPTADINE HYDROCHLORIDE,2010110126,NaN,NaN,...,62033-0346,NJ82J0F8QC,Cyproheptadine Hydrochloride,Cyproheptadine,2YHB6175DO,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,f90faaa3-f40f-4653-8fb8-144d61c6b1d4,cyproheptadine
17740,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,077525,gabapentin,GABAPENTIN,20220321,Anti-epileptic Agent,1993.0,...,71610-069,6CW7F3G59X,GABAPENTIN,GABAPENTIN,6CW7F3G59X,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=7790d906-af26-4e60-bc6f-e906ec03cc24,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=7790d906-af26-4e60-bc6f-e906ec03cc24,7790d906-af26-4e60-bc6f-e906ec03cc24,gabapentin
22866,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,078106,levetiracetam,LEVETIRACETAM,20220108,NaN,1999.0,...,43547-221; 43547-222; 43547-223; 43547-224,44YRR34555,LEVETIRACETAM,LEVETIRACETAM,44YRR34555,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=dd713e60-478c-43cd-b680-ff594a0b3864,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=dd713e60-478c-43cd-b680-ff594a0b3864,dd713e60-478c-43cd-b680-ff594a0b3864,levetiracetam
5728,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,ANDA,077415,bupropion hydrochloride,BUPROPION HYDROCHLORIDE,20220101,Aminoketone,1985.0,...,63187-713,ZG7E5POY8O,BUPROPION HYDROCHLORIDE,BUPROPION,01ZG3TPX31,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=e57451fd-aba5-47d6-9892-ffe2a6021b6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=e57451fd-aba5-47d6-9892-ffe2a6021b6f,e57451fd-aba5-47d6-9892-ffe2a6021b6f,bupropion
17439,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION",INTRAVENOUS,ANDA,204025,fosaprepitant,FOSAPREPITANT,20211231,Substance P/Neurokinin-1 Receptor Antagonist,2008.0,...,16729-240,D35FM8T64X,FOSAPREPITANT DIMEGLUMINE,APREPITANT,1NF15YR6UY,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=4a3bac1c-1702-4963-b967-f1699f03cc6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=4a3bac1c-1702-4963-b967-f1699f03cc6f,4a3bac1c-1702-4963-b967-f1699f03cc6f,fosaprepitant


In [225]:
fda_df['webscraping_name'] = fda_df['Trade Name'].str.extract('(\w+)')

In [310]:
drug_name = {}
sfx = {'webscraping_name':[],'Side Effects':[]}
common_sfx = []
other_sfx = []
drug_side_efx = pd.DataFrame()
for drug in list(fda_df['webscraping_name'][0:10]):
    URL = f'https://www.drugs.com/sfx/{drug}-side-effects.html'
    res = requests.get(URL)
    soup_obj = BS(res.text)
    drug_name = {'webscraping_name':f'{drug}'}
    side_effect = soup_obj.find('p', attrs = {'class':False})
    side_eff = side_effect.text
    #side_efx = side_eff.split('.')
#     common_sfx = side_efx[0]
#     other_sfx = side_efx[1]
#     common_sfx= common_sfx.split(':')
#     other_sfx = other_sfx.split(':')
#     common_sfx_dict = Convert(common_sfx)
#     other_sfx_dict = Convert(other_sfx)
#     common_sfx_dict.update(other_sfx_dict)
#     common_sfx_dict.update(drug_name)
    sfx['webscraping_name'].append(f'{drug}')
    sfx['Side Effects'].append(side_eff)
     
    
    

In [311]:
sfx_df = pd.DataFrame(sfx)

In [313]:
sfx_df

,webscraping_name,Side Effects
0,cyproheptadine,"Applies to cyproheptadine: oral solution, oral tablets"
1,gabapentin,"Commonly reported side effects of gabapentin include: ataxia, dizziness, drowsiness, fatigue, fever, nystagmus disorder, sedated state, and viral infection. Other side effects include: blurred vision, diplopia, peripheral edema, tremor, amblyopia, irritability, and xerostomia. See below for a comprehensive list of adverse effects."
2,levetiracetam,"Commonly reported side effects of levetiracetam include: infection, neurosis, drowsiness, asthenia, headache, nasopharyngitis, nervousness, abnormal behavior, aggressive behavior, agitation, anxiety, apathy, depersonalization, depression, fatigue, hostility, hyperkinetic muscle activity, mental disorders, outbursts of anger, personality disorder, emotional lability, irritability, laceration, and mood changes. Other side effects include: tonic clonic epilepsy, dizziness, vertigo, decreased neutrophils, depressed mood, influenza, neck pain, and pain. See below for a comprehensive list of adverse effects."
3,bupropion,"Commonly reported side effects of bupropion include: insomnia, nausea, pharyngitis, weight loss, constipation, dizziness, headache, and xerostomia. Other side effects include: abdominal pain, agitation, arthralgia, chest pain, migraine, skin rash, urinary frequency, anxiety, asthenia, confusion, diarrhea, hostility, hypertension, lack of concentration, myalgia, nervousness, palpitations, pruritus, tinnitus, tremor, vomiting, anorexia, diaphoresis, dysgeusia, flushing, and abnormal dreams. See below for a comprehensive list of adverse effects."
4,fosaprepitant,Applies to fosaprepitant: intravenous powder for solution
5,paclitaxel,"Commonly reported side effects of paclitaxel include: hypotension, nausea and vomiting, arthralgia, myalgia, alopecia, diarrhea, hypersensitivity condition, mucositis, skin rash, ecg abnormality, increased serum alkaline phosphatase, increased serum aspartate aminotransferase, infusion site reaction, peripheral neuropathy, and flushing. Other side effects include: severe arthralgia, severe myalgia, and increased serum bilirubin. See below for a comprehensive list of adverse effects."
6,teflaro,Common side effects of Teflaro include: positive direct coombs test. Other side effects include: diarrhea. See below for a comprehensive list of adverse effects.
7,dimethyl,"Sorry, the page you were looking could not be found or is unavailable. If you believe this page should exist or if the problem persists, please contact us to report the issue."
8,aricept,"Common side effects of Aricept include: diarrhea, insomnia, and nausea. Other side effects include: fatigue, muscle cramps, vomiting, weight loss, anorexia, and ecchymoses. See below for a comprehensive list of adverse effects."
9,pravastatin,"More frequently reported side effects include: increased serum alanine aminotransferase, skin rash, and increased creatine phosphokinase in blood specimen. See below for a comprehensive list of adverse effects."


In [305]:
sfx_df[sfx_df['Side Effects'].str.contains('Applies')]

,webscraping_name,Side Effects
0,cyproheptadine,"Applies to cyproheptadine: oral solution, oral tablets"
4,fosaprepitant,Applies to fosaprepitant: intravenous powder for solution
13,licart,"Applies to diclofenac topical: topical application cream, topical application gel/jelly, topical application patch extended release, topical application solution, topical application spray"
14,valacyclovir,Applies to valacyclovir: oral tablet
15,phentermine,"Applies to phentermine: oral capsule, oral tablet"
...,...,...
5553,advair,"Applies to fluticasone / salmeterol: inhalation aerosol liquid, inhalation disk, inhalation powder"
5557,neostigmine,Applies to neostigmine: injection solution
5560,methylprednisolone,Applies to methylprednisolone: oral tablet
5561,rifampin,Applies to rifampin: oral capsule


In [283]:
URL = 'https://www.drugs.com/sfx/cyproheptadine-side-effects.html'
res = requests.get(URL)
soup_obj = BS(res.text)

In [295]:
soup_obj.find_all('p', text = True)

[<p>Side effects include:</p>,
 <p><i>Applies to cyproheptadine: compounding powder, oral syrup, oral tablet</i></p>,
 <p id="ref_1">1. Cerner Multum,  Inc. "UK Summary of Product Characteristics." O 0</p>,
 <p id="ref_2">2. "Product Information. Periactin (cyproheptadine)." Merck &amp; Co, Inc, West Point, PA. </p>,
 <p class="ddc-disclaimer">Always consult your healthcare provider to ensure the information displayed on this page applies to your personal circumstances.</p>,
 <p class="ddc-disclaimer-link"><a href="/support/disclaimer.html">Medical Disclaimer</a></p>,
 <p>Loading...</p>,
 <p>Loading...</p>,
 <p>The easiest way to lookup drug information, identify pills, check interactions and set up your own personal medication records. Available for Android and iOS devices.</p>]

In [236]:
common_sfx_dict

{'Commonly reported side effects of gabapentin include': ' ataxia, dizziness, drowsiness, fatigue, fever, nystagmus disorder, sedated state, and viral infection',
 ' Other side effects include': ' blurred vision, diplopia, peripheral edema, tremor, amblyopia, irritability, and xerostomia',
 'webscraping_name': 'pravastatin'}

In [16]:
adverse_rxn.head()

,0,1,SET ID
0,Adverse Reaction,Open-Label Titration Period + Stable Dose Period (up to 16 weeks) (N=154) %,1e0ae43a-037f-42af-8e23-a0e51d75abe8
1,Nausea,21,1e0ae43a-037f-42af-8e23-a0e51d75abe8
2,Headache,16,1e0ae43a-037f-42af-8e23-a0e51d75abe8
3,Anxiety*,12,1e0ae43a-037f-42af-8e23-a0e51d75abe8
4,Dizziness,12,1e0ae43a-037f-42af-8e23-a0e51d75abe8


In [17]:
food_df = pd.read_csv('Data/food.csv')

In [18]:
food_nutrient_df = pd.read_csv('Data/food_nutrient.csv')

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
nutrient_df = pd.read_csv('Data/nutrient.csv').rename(columns = {'id':'nutrient_id'})

In [20]:
food_and_nutrient = pd.merge(food_nutrient_df, nutrient_df, on = 'nutrient_id')

In [21]:
food_and_nutrient.head()

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name,nutrient_nbr,rank
0,13338591,323505,2048,35.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
1,13338389,326196,2048,36.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,13338390,747447,2048,32.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
3,13338391,321611,2048,20.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
4,13338392,746769,2048,17.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0


In [22]:
food = pd.merge(food_df,food_and_nutrient, on = 'fdc_id')

In [23]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date,id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name,nutrient_nbr,rank
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",NaN,2019-04-01,1283674,1003,5.88,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,Protein,G,203.0,600.0
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",NaN,2019-04-01,1283675,1007,3.50,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,Ash,G,207.0,1000.0
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",NaN,2019-04-01,1283676,1062,1286.00,0.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy,kJ,268.0,400.0
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",NaN,2019-04-01,1283677,1079,1.20,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fiber, total dietary",G,291.0,1200.0
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",NaN,2019-04-01,1283678,1089,2.12,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,"Iron, Fe",MG,303.0,5400.0


In [24]:
food = food.drop(columns = ['food_category_id','min','max','median','loq', 'footnote', 'min_year_acquired'])

In [77]:
food.head()

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",2019-04-01,1283674,1003,5.88,1.0,46.0,Protein,G,203.0,600.0
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",2019-04-01,1283675,1007,3.50,1.0,46.0,Ash,G,207.0,1000.0
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",2019-04-01,1283676,1062,1286.00,0.0,49.0,Energy,kJ,268.0,400.0
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",2019-04-01,1283677,1079,1.20,1.0,46.0,"Fiber, total dietary",G,291.0,1200.0
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",2019-04-01,1283678,1089,2.12,1.0,46.0,"Iron, Fe",MG,303.0,5400.0


In [25]:
#se_df = pd.read_table('Data/meddra_all_label_se.tsv')

In [26]:
#drug_code = pd.read_table('Data/drug_atc.tsv')

In [27]:
#drug_names = pd.read_table('Data/drug_names.tsv')

In [30]:
#drug_names.head(10)

,CID100000085,carnitine
0,CID100000119,gamma-aminobutyric
1,CID100000137,5-aminolevulinic
2,CID100000143,leucovorin
3,CID100000146,5-methyltetrahydrofolate
4,CID100000158,PGE2
5,CID100000159,prostacyclin
6,CID100000160,prostaglandin
7,CID100000175,acetate
8,CID100000187,acetylcholine
9,CID100000191,adenosine


In [29]:
fda_df[fda_df['Trade Name']=='CARNITINE']

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID


In [331]:
THISURL = 'https://www.drugs.com/alpha/condition/a'

In [332]:
res1 = requests.get(THISURL)

In [333]:
soup_1 = BS(res1.text)

In [346]:
conditions = soup_1.select('.ddc-list-column-2 a')

In [370]:
conditions_list = list(conditions)

In [397]:
re.search(r'(>\w+\s?\w+?<)', str(conditions_list[1]))

<re.Match object; span=(67, 76), match='>Abscess<'>

In [391]:
conditions_list

[<a href="/cg/abdominal-pain.html" onclick="DDC_recordConditionHit(event);">Abdominal Pain</a>,
 <a href="/cg/abscess.html" onclick="DDC_recordConditionHit(event);">Abscess</a>,
 <a href="/cg/abuse-of-alcohol.html" onclick="DDC_recordConditionHit(event);">Abuse of Alcohol</a>,
 <a href="/condition/acidosis.html" onclick="DDC_recordConditionHit(event);">Acidosis</a>,
 <a href="/cg/acne.html" onclick="DDC_recordConditionHit(event);">Acne</a>,
 <a href="/cg/acute-cough.html" onclick="DDC_recordConditionHit(event);">Acute Cough</a>,
 <a href="/health-guide/acute-myeloid-leukemia-aml.html" onclick="DDC_recordConditionHit(event);">Acute Myeloid Leukemia (AML)</a>,
 <a href="/health-guide/acute-sinusitis.html" onclick="DDC_recordConditionHit(event);">Acute Sinusitis</a>,
 <a href="/cg/adhd-in-adults.html" onclick="DDC_recordConditionHit(event);">ADHD</a>,
 <a href="/cg/adhd-in-children.html" onclick="DDC_recordConditionHit(event);">ADHD in Children</a>,
 <a href="/cg/aids.html" onclick="DDC_r

In [401]:
x = len(conditions_list)
cond_list = []
for i in range(0,x):
    cond_list.append(re.search(r'(>\w+\W?\s?\w+?\s?\w+?\s?\W?\w+?\W?<)', str(conditions_list[i])).group(1))
    condition_list.append(cond_list[i].replace('>','')
    cond_list[i].replace('<','')

In [402]:
cond_list

['>Abdominal Pain<',
 '>Abscess<',
 '>Abuse of Alcohol<',
 '>Acidosis<',
 '>Acne<',
 '>Acute Cough<',
 '>Acute Myeloid Leukemia (AML)<',
 '>Acute Sinusitis<',
 '>ADHD<',
 '>ADHD in Children<',
 '>AIDS<',
 '>Allergies<',
 '>Allergies <',
 '>Alopecia<',
 '>Amnesia<',
 '>Anemia<',
 '>Angina<',
 '>Anxiety<',
 '>Arrhythmia<',
 '>Arthritis<',
 '>Asthma, Acute<',
 '>Atrial Fibrillation<',
 '>Autism<',
 '>Autoimmune Disorders<']